In [15]:
import torch
import numpy as np
from datetime import datetime

## Load data

In [20]:
print('loading velocity data')
start_time = datetime.now()
velocity_data = np.genfromtxt('./data/snapshots_on_grid_Velocity_1.csv', delimiter=',')
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print('loading pressure data')
start_time = datetime.now()
pressure_data = np.genfromtxt('./data/snapshots_on_grid_Pressure_1.csv', delimiter=',')
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

loading velocity data
Duration: 0:00:43.384693
loading pressure data
Duration: 0:00:20.446308


## POD approach

In [43]:
def POD(snapshots_matrix, nPOD, nDim):
    singular_values = []
    nx = 221
    ny = 42
    nz = 1
    nrows, ncols = snapshots_matrix.shape
    if nrows > ncols:
        SSmatrix = np.dot(snapshots_matrix.T, snapshots_matrix)
    else:
        SSmatrix = np.dot(snapshots_matrix, snapshots_matrix.T)
        print('WARNING - CHECK HOW THE BASIS FUNCTIONS ARE CALCULATED WITH THIS METHOD')
    print('SSmatrix', SSmatrix.shape)
    #print('SSmatrix', SSmatrix)
    eigvalues, v = np.linalg.eigh(SSmatrix)
    eigvalues =  eigvalues[::-1]
    # get rid of small negative eigenvalues
    eigvalues[eigvalues<0] = 0
    s_values = np.sqrt(eigvalues)

    singular_values.append(s_values)

    cumulative_info = np.zeros(len(eigvalues))
    for j in range(len(eigvalues)):
        if j==0:
            cumulative_info[j] = eigvalues[j]
        else: 
            cumulative_info[j] = cumulative_info[j-1] + eigvalues[j]

    cumulative_info = cumulative_info / cumulative_info[-1]
    nAll = len(eigvalues)
    basis_functions = np.zeros((nx*ny*nz*nDim,nPOD))
    print ("retaining", nPOD, "basis functions of a possible", len(eigvalues))
    for j in reversed(range(nAll-nPOD,nAll)):
        Av = np.dot(snapshots_matrix,v[:,j])
        basis_functions[:,nAll-j-1] = Av/np.linalg.norm(Av)
    return basis_functions

## Save basis functions

In [44]:
velocity_basis = POD(snapshots_matrix=velocity_data, nPOD=10, nDim=2)
np.savetxt('./data/Velocity_basis.csv', velocity_basis, delimiter=',')

SSmatrix (2000, 2000)
retaining 10 basis functions of a possible 2000


In [47]:
pressure_basis = POD(snapshots_matrix=pressure_data, nPOD=10, nDim=1)
np.savetxt('./data/Pressure_basis.csv', pressure_basis, delimiter=',')

SSmatrix (2000, 2000)
retaining 10 basis functions of a possible 2000
